# Devoir python

## 2_2_Mini Projets Python Cybersécurité 1

### Projet 0

In [ ]:
import os
import shutil
import logging

# Configuration de la journalisation (log)
logging.basicConfig(
    filename="classification_log.txt",
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)

def classer_fichiers(repertoire_source):
    try:
        if not os.path.exists(repertoire_source):
            raise FileNotFoundError(f"Le répertoire source '{repertoire_source}' n'existe pas.")

        # Parcourir tous les fichiers du répertoire source
        for fichier in os.listdir(repertoire_source):
            chemin_complet = os.path.join(repertoire_source, fichier)

            # Ignorer les dossiers
            if os.path.isdir(chemin_complet):
                continue

            # Identifier l'extension du fichier
            _, extension = os.path.splitext(fichier)
            extension = extension.lstrip(".").lower()  # Retire le point et met en minuscule

            if not extension:  # Si le fichier n'a pas d'extension
                extension = "sans_extension"

            # Créer le sous-dossier correspondant s'il n'existe pas
            sous_dossier = os.path.join(repertoire_source, extension)
            if not os.path.exists(sous_dossier):
                os.makedirs(sous_dossier)

            # Déplacer le fichier dans le sous-dossier
            destination = os.path.join(sous_dossier, fichier)
            shutil.move(chemin_complet, destination)

            # Log de l'action effectuée
            logging.info(f"Fichier déplacé : {chemin_complet} -> {destination}")

    except Exception as e:
        logging.error(f"Erreur lors de la classification : {e}")

# Demander le répertoire source à l'utilisateur
repertoire_source = input("Entrez le chemin du répertoire source : ")
classer_fichiers(repertoire_source)

print("Classification terminée. Consultez 'classification_log.txt' pour les détails.")


### Projet 1

In [ ]:
import os
import shutil
import logging
import pandas as pd
import re
import matplotlib.pyplot as plt

# Configuration de la journalisation (log)
logging.basicConfig(
    filename="classification_log.txt",
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)

def classer_fichiers(repertoire_source):
    try:
        if not os.path.exists(repertoire_source):
            raise FileNotFoundError(f"Le répertoire source '{repertoire_source}' n'existe pas.")

        # Parcourir tous les fichiers du répertoire source
        for fichier in os.listdir(repertoire_source):
            chemin_complet = os.path.join(repertoire_source, fichier)

            # Ignorer les dossiers
            if os.path.isdir(chemin_complet):
                continue

            # Identifier l'extension du fichier
            _, extension = os.path.splitext(fichier)
            extension = extension.lstrip(".").lower()  # Retire le point et met en minuscule

            if not extension:  # Si le fichier n'a pas d'extension
                extension = "sans_extension"

            # Créer le sous-dossier correspondant s'il n'existe pas
            sous_dossier = os.path.join(repertoire_source, extension)
            if not os.path.exists(sous_dossier):
                os.makedirs(sous_dossier)

            # Déplacer le fichier dans le sous-dossier
            destination = os.path.join(sous_dossier, fichier)
            shutil.move(chemin_complet, destination)

            # Log de l'action effectuée
            logging.info(f"Fichier déplacé : {chemin_complet} -> {destination}")

    except Exception as e:
        logging.error(f"Erreur lors de la classification : {e}")

def analyser_logs_acces(fichier_logs):
    try:
        # Lire le fichier de logs
        logs = pd.read_csv(fichier_logs, sep=' ', header=None, names=["IP", "Date", "Requete", "Status"])

        # Analyser les IP les plus fréquentes
        frequent_ips = logs["IP"].value_counts().head(10)

        # Visualisation
        frequent_ips.plot(kind="bar", title="Top 10 des IP les plus actives", xlabel="Adresse IP", ylabel="Nombre de requêtes")
        plt.show()

        logging.info("Analyse des logs réussie.")

    except Exception as e:
        logging.error(f"Erreur lors de l'analyse des logs : {e}")

def scanner_ports(ip, ports):
    import socket

    resultats = {}
    for port in ports:
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            s.settimeout(1)
            if s.connect_ex((ip, port)) == 0:
                resultats[port] = "Ouvert"
            else:
                resultats[port] = "Fermé"

    # Visualisation
    ouvert = [p for p, status in resultats.items() if status == "Ouvert"]
    plt.bar(resultats.keys(), [1 if port in ouvert else 0 for port in resultats.keys()])
    plt.title(f"Ports scannés pour {ip}")
    plt.xlabel("Ports")
    plt.ylabel("Status (1=Ouvert, 0=Fermé)")
    plt.show()

    return resultats

# Demander le répertoire source à l'utilisateur
repertoire_source = input("Entrez le chemin du répertoire source : ")
classer_fichiers(repertoire_source)

print("Classification terminée. Consultez 'classification_log.txt' pour les détails.")

# Exemple pour analyser les logs
fichier_logs = input("Entrez le chemin du fichier de logs : ")
analyser_logs_acces(fichier_logs)

# Exemple pour scanner des ports
ip_cible = input("Entrez l'adresse IP cible : ")
ports = list(range(20, 25))  # Exemple de ports à scanner
resultats_ports = scanner_ports(ip_cible, ports)
print(resultats_ports)


### Projet 3

In [ ]:
import os
import shutil
import logging
import pandas as pd
import re
import matplotlib.pyplot as plt
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler

# Configuration de la journalisation (log)
logging.basicConfig(
    filename="classification_log.txt",
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)

class SurveillanceHandler(FileSystemEventHandler):
    def on_modified(self, event):
        if not event.is_directory:
            logging.info(f"Fichier modifié : {event.src_path}")

    def on_created(self, event):
        if not event.is_directory:
            logging.info(f"Fichier créé : {event.src_path}")

    def on_deleted(self, event):
        if not event.is_directory:
            logging.info(f"Fichier supprimé : {event.src_path}")


def surveiller_repertoire(repertoire):
    try:
        if not os.path.exists(repertoire):
            raise FileNotFoundError(f"Le répertoire '{repertoire}' n'existe pas.")

        event_handler = SurveillanceHandler()
        observer = Observer()
        observer.schedule(event_handler, path=repertoire, recursive=True)
        observer.start()

        print(f"Surveillance en cours dans le répertoire : {repertoire}")
        print("Appuyez sur Ctrl+C pour arrêter.")

        try:
            while True:
                pass
        except KeyboardInterrupt:
            observer.stop()
        observer.join()

    except Exception as e:
        logging.error(f"Erreur lors de la surveillance : {e}")

def classer_fichiers(repertoire_source):
    try:
        if not os.path.exists(repertoire_source):
            raise FileNotFoundError(f"Le répertoire source '{repertoire_source}' n'existe pas.")

        # Parcourir tous les fichiers du répertoire source
        for fichier in os.listdir(repertoire_source):
            chemin_complet = os.path.join(repertoire_source, fichier)

            # Ignorer les dossiers
            if os.path.isdir(chemin_complet):
                continue

            # Identifier l'extension du fichier
            _, extension = os.path.splitext(fichier)
            extension = extension.lstrip(".").lower()  # Retire le point et met en minuscule

            if not extension:  # Si le fichier n'a pas d'extension
                extension = "sans_extension"

            # Créer le sous-dossier correspondant s'il n'existe pas
            sous_dossier = os.path.join(repertoire_source, extension)
            if not os.path.exists(sous_dossier):
                os.makedirs(sous_dossier)

            # Déplacer le fichier dans le sous-dossier
            destination = os.path.join(sous_dossier, fichier)
            shutil.move(chemin_complet, destination)

            # Log de l'action effectuée
            logging.info(f"Fichier déplacé : {chemin_complet} -> {destination}")

    except Exception as e:
        logging.error(f"Erreur lors de la classification : {e}")

def analyser_logs_acces(fichier_logs):
    try:
        # Lire le fichier de logs
        logs = pd.read_csv(fichier_logs, sep=' ', header=None, names=["IP", "Date", "Requete", "Status"])

        # Analyser les IP les plus fréquentes
        frequent_ips = logs["IP"].value_counts().head(10)

        # Visualisation
        frequent_ips.plot(kind="bar", title="Top 10 des IP les plus actives", xlabel="Adresse IP", ylabel="Nombre de requêtes")
        plt.show()

        logging.info("Analyse des logs réussie.")

    except Exception as e:
        logging.error(f"Erreur lors de l'analyse des logs : {e}")

def scanner_ports(ip, ports):
    import socket

    resultats = {}
    for port in ports:
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            s.settimeout(1)
            if s.connect_ex((ip, port)) == 0:
                resultats[port] = "Ouvert"
            else:
                resultats[port] = "Fermé"

    # Visualisation
    ouvert = [p for p, status in resultats.items() if status == "Ouvert"]
    plt.bar(resultats.keys(), [1 if port in ouvert else 0 for port in resultats.keys()])
    plt.title(f"Ports scannés pour {ip}")
    plt.xlabel("Ports")
    plt.ylabel("Status (1=Ouvert, 0=Fermé)")
    plt.show()

    return resultats

# Demander le répertoire source à l'utilisateur
repertoire_source = input("Entrez le chemin du répertoire source : ")
classer_fichiers(repertoire_source)

print("Classification terminée. Consultez 'classification_log.txt' pour les détails.")

# Exemple pour analyser les logs
fichier_logs = input("Entrez le chemin du fichier de logs : ")
analyser_logs_acces(fichier_logs)

# Exemple pour scanner des ports
ip_cible = input("Entrez l'adresse IP cible : ")
ports = list(range(20, 25))  # Exemple de ports à scanner
resultats_ports = scanner_ports(ip_cible, ports)
print(resultats_ports)

# Exemple pour surveiller un répertoire
repertoire_surveillance = input("Entrez le répertoire à surveiller : ")
surveiller_repertoire(repertoire_surveillance)


### Projet 4

In [ ]:
import os
import shutil
import logging
import pandas as pd
import re
import matplotlib.pyplot as plt
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler

# Configuration de la journalisation (log)
logging.basicConfig(
    filename="classification_log.txt",
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)

class SurveillanceHandler(FileSystemEventHandler):
    def on_modified(self, event):
        if not event.is_directory:
            logging.info(f"Fichier modifié : {event.src_path}")

    def on_created(self, event):
        if not event.is_directory:
            logging.info(f"Fichier créé : {event.src_path}")

    def on_deleted(self, event):
        if not event.is_directory:
            logging.info(f"Fichier supprimé : {event.src_path}")


def surveiller_repertoire(repertoire):
    try:
        if not os.path.exists(repertoire):
            raise FileNotFoundError(f"Le répertoire '{repertoire}' n'existe pas.")

        event_handler = SurveillanceHandler()
        observer = Observer()
        observer.schedule(event_handler, path=repertoire, recursive=True)
        observer.start()

        print(f"Surveillance en cours dans le répertoire : {repertoire}")
        print("Appuyez sur Ctrl+C pour arrêter.")

        try:
            while True:
                pass
        except KeyboardInterrupt:
            observer.stop()
        observer.join()

    except Exception as e:
        logging.error(f"Erreur lors de la surveillance : {e}")

def classer_fichiers(repertoire_source):
    try:
        if not os.path.exists(repertoire_source):
            raise FileNotFoundError(f"Le répertoire source '{repertoire_source}' n'existe pas.")

        # Parcourir tous les fichiers du répertoire source
        for fichier in os.listdir(repertoire_source):
            chemin_complet = os.path.join(repertoire_source, fichier)

            # Ignorer les dossiers
            if os.path.isdir(chemin_complet):
                continue

            # Identifier l'extension du fichier
            _, extension = os.path.splitext(fichier)
            extension = extension.lstrip(".").lower()  # Retire le point et met en minuscule

            if not extension:  # Si le fichier n'a pas d'extension
                extension = "sans_extension"

            # Créer le sous-dossier correspondant s'il n'existe pas
            sous_dossier = os.path.join(repertoire_source, extension)
            if not os.path.exists(sous_dossier):
                os.makedirs(sous_dossier)

            # Déplacer le fichier dans le sous-dossier
            destination = os.path.join(sous_dossier, fichier)
            shutil.move(chemin_complet, destination)

            # Log de l'action effectuée
            logging.info(f"Fichier déplacé : {chemin_complet} -> {destination}")

    except Exception as e:
        logging.error(f"Erreur lors de la classification : {e}")

def analyser_logs_acces(fichier_logs):
    try:
        # Lire le fichier de logs
        logs = pd.read_csv(fichier_logs, sep=' ', header=None, names=["IP", "Date", "Requete", "Status"])

        # Analyser les IP les plus fréquentes
        frequent_ips = logs["IP"].value_counts().head(10)

        # Visualisation
        frequent_ips.plot(kind="bar", title="Top 10 des IP les plus actives", xlabel="Adresse IP", ylabel="Nombre de requêtes")
        plt.show()

        logging.info("Analyse des logs réussie.")

    except Exception as e:
        logging.error(f"Erreur lors de l'analyse des logs : {e}")

def scanner_ports(ip, ports):
    import socket

    resultats = {}
    for port in ports:
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            s.settimeout(1)
            if s.connect_ex((ip, port)) == 0:
                resultats[port] = "Ouvert"
            else:
                resultats[port] = "Fermé"

    # Visualisation
    ouvert = [p for p, status in resultats.items() if status == "Ouvert"]
    plt.bar(resultats.keys(), [1 if port in ouvert else 0 for port in resultats.keys()])
    plt.title(f"Ports scannés pour {ip}")
    plt.xlabel("Ports")
    plt.ylabel("Status (1=Ouvert, 0=Fermé)")
    plt.show()

    return resultats

def detecter_phishing(fichier_emails):
    try:
        with open(fichier_emails, 'r') as f:
            emails = f.readlines()

        phishing_signes = []
        for email in emails:
            exp_suspect = re.search(r'(noreply|admin|support)@.*', email)
            lien_suspect = re.search(r'http[s]?://[^"]*', email)

            if exp_suspect or lien_suspect:
                phishing_signes.append(email)

        print("Emails suspects détectés :")
        for signe in phishing_signes:
            print(signe)

        logging.info(f"Analyse des emails terminée, {len(phishing_signes)} emails suspects trouvés.")

    except Exception as e:
        logging.error(f"Erreur lors de la détection de phishing : {e}")

# Demander le répertoire source à l'utilisateur
repertoire_source = input("Entrez le chemin du répertoire source : ")
classer_fichiers(repertoire_source)

print("Classification terminée. Consultez 'classification_log.txt' pour les détails.")

# Exemple pour analyser les logs
fichier_logs = input("Entrez le chemin du fichier de logs : ")
analyser_logs_acces(fichier_logs)

# Exemple pour scanner des ports
ip_cible = input("Entrez l'adresse IP cible : ")
ports = list(range(20, 25))  # Exemple de ports à scanner
resultats_ports = scanner_ports(ip_cible, ports)
print(resultats_ports)

# Exemple pour surveiller un répertoire
repertoire_surveillance = input("Entrez le répertoire à surveiller : ")
surveiller_repertoire(repertoire_surveillance)

# Exemple pour détecter des emails suspects
fichier_emails = input("Entrez le chemin du fichier d'emails : ")
detecter_phishing(fichier_emails)


### Projet 5

In [ ]:
import os
import shutil
import logging
import pandas as pd
import re
import matplotlib.pyplot as plt
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler
import itertools
import time

# Configuration de la journalisation (log)
logging.basicConfig(
    filename="classification_log.txt",
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)

class SurveillanceHandler(FileSystemEventHandler):
    def on_modified(self, event):
        if not event.is_directory:
            logging.info(f"Fichier modifié : {event.src_path}")

    def on_created(self, event):
        if not event.is_directory:
            logging.info(f"Fichier créé : {event.src_path}")

    def on_deleted(self, event):
        if not event.is_directory:
            logging.info(f"Fichier supprimé : {event.src_path}")


def surveiller_repertoire(repertoire):
    try:
        if not os.path.exists(repertoire):
            raise FileNotFoundError(f"Le répertoire '{repertoire}' n'existe pas.")

        event_handler = SurveillanceHandler()
        observer = Observer()
        observer.schedule(event_handler, path=repertoire, recursive=True)
        observer.start()

        print(f"Surveillance en cours dans le répertoire : {repertoire}")
        print("Appuyez sur Ctrl+C pour arrêter.")

        try:
            while True:
                pass
        except KeyboardInterrupt:
            observer.stop()
        observer.join()

    except Exception as e:
        logging.error(f"Erreur lors de la surveillance : {e}")

def classer_fichiers(repertoire_source):
    try:
        if not os.path.exists(repertoire_source):
            raise FileNotFoundError(f"Le répertoire source '{repertoire_source}' n'existe pas.")

        # Parcourir tous les fichiers du répertoire source
        for fichier in os.listdir(repertoire_source):
            chemin_complet = os.path.join(repertoire_source, fichier)

            # Ignorer les dossiers
            if os.path.isdir(chemin_complet):
                continue

            # Identifier l'extension du fichier
            _, extension = os.path.splitext(fichier)
            extension = extension.lstrip(".").lower()  # Retire le point et met en minuscule

            if not extension:  # Si le fichier n'a pas d'extension
                extension = "sans_extension"

            # Créer le sous-dossier correspondant s'il n'existe pas
            sous_dossier = os.path.join(repertoire_source, extension)
            if not os.path.exists(sous_dossier):
                os.makedirs(sous_dossier)

            # Déplacer le fichier dans le sous-dossier
            destination = os.path.join(sous_dossier, fichier)
            shutil.move(chemin_complet, destination)

            # Log de l'action effectuée
            logging.info(f"Fichier déplacé : {chemin_complet} -> {destination}")

    except Exception as e:
        logging.error(f"Erreur lors de la classification : {e}")

def analyser_logs_acces(fichier_logs):
    try:
        # Lire le fichier de logs
        logs = pd.read_csv(fichier_logs, sep=' ', header=None, names=["IP", "Date", "Requete", "Status"])

        # Analyser les IP les plus fréquentes
        frequent_ips = logs["IP"].value_counts().head(10)

        # Visualisation
        frequent_ips.plot(kind="bar", title="Top 10 des IP les plus actives", xlabel="Adresse IP", ylabel="Nombre de requêtes")
        plt.show()

        logging.info("Analyse des logs réussie.")

    except Exception as e:
        logging.error(f"Erreur lors de l'analyse des logs : {e}")

def scanner_ports(ip, ports):
    import socket

    resultats = {}
    for port in ports:
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            s.settimeout(1)
            if s.connect_ex((ip, port)) == 0:
                resultats[port] = "Ouvert"
            else:
                resultats[port] = "Fermé"

    # Visualisation
    ouvert = [p for p, status in resultats.items() if status == "Ouvert"]
    plt.bar(resultats.keys(), [1 if port in ouvert else 0 for port in resultats.keys()])
    plt.title(f"Ports scannés pour {ip}")
    plt.xlabel("Ports")
    plt.ylabel("Status (1=Ouvert, 0=Fermé)")
    plt.show()

    return resultats

def detecter_phishing(fichier_emails):
    try:
        with open(fichier_emails, 'r') as f:
            emails = f.readlines()

        phishing_signes = []
        for email in emails:
            exp_suspect = re.search(r'(noreply|admin|support)@.*', email)
            lien_suspect = re.search(r'http[s]?://[^"]*', email)

            if exp_suspect or lien_suspect:
                phishing_signes.append(email)

        print("Emails suspects détectés :")
        for signe in phishing_signes:
            print(signe)

        logging.info(f"Analyse des emails terminée, {len(phishing_signes)} emails suspects trouvés.")

    except Exception as e:
        logging.error(f"Erreur lors de la détection de phishing : {e}")

def bruteforce_mdp(mot_de_passe, dictionnaire):
    try:
        for tentative in dictionnaire:
            print(f"Tentative : {tentative.strip()}")
            if tentative.strip() == mot_de_passe:
                print(f"Mot de passe trouvé : {tentative.strip()}")
                logging.info(f"Mot de passe trouvé : {tentative.strip()}")
                return True
            time.sleep(0.5)  # Pause pour simuler un délai

        print("Mot de passe non trouvé dans le dictionnaire.")
        logging.warning("Mot de passe non trouvé dans le dictionnaire.")
        return False

    except Exception as e:
        logging.error(f"Erreur lors du bruteforce : {e}")

# Demander le répertoire source à l'utilisateur
repertoire_source = input("Entrez le chemin du répertoire source : ")
classer_fichiers(repertoire_source)

print("Classification terminée. Consultez 'classification_log.txt' pour les détails.")

# Exemple pour analyser les logs
fichier_logs = input("Entrez le chemin du fichier de logs : ")
analyser_logs_acces(fichier_logs)

# Exemple pour scanner des ports
ip_cible = input("Entrez l'adresse IP cible : ")
ports = list(range(20, 25))  # Exemple de ports à scanner
resultats_ports = scanner_ports(ip_cible, ports)
print(resultats_ports)

# Exemple pour surveiller un répertoire
repertoire_surveillance = input("Entrez le répertoire à surveiller : ")
surveiller_repertoire(repertoire_surveillance)

# Exemple pour détecter des emails suspects
fichier_emails = input("Entrez le chemin du fichier d'emails : ")
detecter_phishing(fichier_emails)

# Exemple pour brute-force des mots de passe
mdp_correct = "password123"
chemin_dictionnaire = input("Entrez le chemin du fichier de dictionnaire : ")
with open(chemin_dictionnaire, 'r') as file:
    dictionnaire = file.readlines()
bruteforce_mdp(mdp_correct, dictionnaire)


### projet 6 et 7

In [ ]:
import os
import shutil
import logging
import pandas as pd
import re
import matplotlib.pyplot as plt
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler
import itertools
import time
import socket
import ssl

# Configuration de la journalisation (log)
logging.basicConfig(
    filename="classification_log.txt",
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)

class SurveillanceHandler(FileSystemEventHandler):
    def on_modified(self, event):
        if not event.is_directory:
            logging.info(f"Fichier modifié : {event.src_path}")

    def on_created(self, event):
        if not event.is_directory:
            logging.info(f"Fichier créé : {event.src_path}")

    def on_deleted(self, event):
        if not event.is_directory:
            logging.info(f"Fichier supprimé : {event.src_path}")


def analyser_logs_securite(fichier_logs):
    try:
        # Lire les logs de sécurité
        logs = pd.read_csv(fichier_logs)

        # Filtrer les tentatives de connexion échouées
        tentatives_echouees = logs[logs['EventID'] == 4625]  # Exemple : EventID 4625 = échec de connexion
        
        # Analyser la fréquence des événements
        frequence_evenements = logs['EventID'].value_counts()

        # Visualisation des types d'événements
        frequence_evenements.plot(kind="bar", title="Fréquence des événements de sécurité", xlabel="EventID", ylabel="Nombre d'occurrences")
        plt.show()

        # Générer un rapport
        logging.info(f"Analyse des logs de sécurité terminée. Tentatives échouées détectées : {len(tentatives_echouees)}")

        # Sauvegarder les tentatives échouées
        tentatives_echouees.to_csv("tentatives_echouees.csv", index=False)

    except Exception as e:
        logging.error(f"Erreur lors de l'analyse des logs de sécurité : {e}")

def verifier_certificats_ssl(domaines):
    certificats = []
    for domaine in domaines:
        try:
            contexte = ssl.create_default_context()
            with socket.create_connection((domaine, 443)) as sock:
                with contexte.wrap_socket(sock, server_hostname=domaine) as ssock:
                    certificat = ssock.getpeercert()

                    # Vérifier la date d'expiration
                    expire = certificat['notAfter']
                    certificats.append({
                        'Domaine': domaine,
                        'Expiration': expire
                    })
                    print(f"Certificat valide pour {domaine} jusqu'à {expire}")
        except Exception as e:
            logging.error(f"Erreur lors de la vérification du certificat pour {domaine} : {e}")

    # Visualisation
    domaines = [cert['Domaine'] for cert in certificats]
    dates = [cert['Expiration'] for cert in certificats]
    plt.bar(domaines, range(len(dates)))
    plt.title("Certificats SSL - Validité")
    plt.xlabel("Domaines")
    plt.ylabel("Validité")
    plt.show()

    return certificats

# Exemple pour analyser les logs de sécurité
fichier_logs_securite = input("Entrez le chemin du fichier de logs de sécurité : ")
analyser_logs_securite(fichier_logs_securite)

# Exemple pour vérifier les certificats SSL
domaines = ["example.com", "google.com"]
certificats = verifier_certificats_ssl(domaines)
print(certificats)
